In [1]:
from pysheds.grid import Grid
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
from skimage.io import imread
import random
import numpy as np
import cv2
from PIL import Image
import os
import h5py

In [2]:
def sketch_rivers(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    depressions = grid.detect_depressions('dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    # Compute flow direction based on corrected DEM
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Compute flow accumulation based on computed flow direction
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    
    downsampled_rivers = cv2.pyrDown(np.log(grid.view('acc') + 1), dstsize=(128, 128))
    upsampled_rivers = cv2.pyrUp(downsampled_rivers, dstsize=(256, 256))
    upsampled_rivers = (upsampled_rivers - np.amin(upsampled_rivers)) / (np.amax(upsampled_rivers) - np.amin(upsampled_rivers))
    upsampled_rivers = np.array(upsampled_rivers * 255, dtype=np.uint8)
    _, thresholded_river = cv2.threshold(upsampled_rivers, 110, 255, cv2.THRESH_BINARY)
    thresholded_river[thresholded_river == 255] = 1
    skeletonized_rivers = skeletonize(thresholded_river)
    rivers_sketch = Image.fromarray(skeletonized_rivers)
    rgb_rivers_sketch = rivers_sketch.convert('RGB')

    basins_sketch = Image.fromarray(depressions)
    rgb_basins_sketch = basins_sketch.convert('RGB')
    
    return rgb_rivers_sketch, rgb_basins_sketch

In [3]:
def sketch_ridges(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.dem = grid.dem.max() - grid.dem
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    peaks = grid.detect_depressions('flooded_dem')
    
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    # Compute flow direction based on corrected DEM
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Compute flow accumulation based on computed flow direction
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    downsampled_ridges = cv2.pyrDown(np.log(grid.view('acc') + 1), dstsize=(128,128))
    
    upsampled_ridges = cv2.pyrUp(downsampled_ridges, dstsize=(256, 256))
    upsampled_ridges = (upsampled_ridges - np.amin(upsampled_ridges)) / (np.amax(upsampled_ridges) - np.amin(upsampled_ridges))
    upsampled_ridges = np.array(upsampled_ridges * 255, dtype=np.uint8)
    _, thresholded_ridges = cv2.threshold(upsampled_ridges, 120, 255, cv2.THRESH_BINARY)
    thresholded_ridges[thresholded_ridges == 255] = 1
    skeletonized_ridges = skeletonize(thresholded_ridges)
    ridges_sketch = Image.fromarray(skeletonized_ridges)
    rgb_ridges_sketch = ridges_sketch.convert('RGB')
    
    peaks_sketch = Image.fromarray(peaks)
    rgb_peaks_sketch = peaks_sketch.convert('RGB')
    
    return rgb_ridges_sketch, rgb_peaks_sketch

In [4]:
def prepare_heightmap_sketch(file_path):
    rivers, basins = sketch_rivers(file_path)
    ridges, peaks = sketch_ridges(file_path)

    dim = (256, 256)

    sketch = np.zeros([256,256,3],dtype=np.uint8)
    sketch.fill(255)
    
    sketch_rivers_flag = True
    sketch_basins_flag = True
    sketch_ridges_flag = True
    sketch_peaks_flag = True
    '''
    if random.randrange(4) == 0:
        sketch_rivers_flag = True
    
    if random.randrange(4) == 0:
        sketch_basins_flag = True
        
    if random.randrange(4) == 0:
        sketch_ridges_flag = True
        
    if random.randrange(4) == 0:
        sketch_peaks_flag = True
               
    if sketch_rivers_flag == False and sketch_basins_flag == False and sketch_ridges_flag == False and sketch_peaks_flag == False:
        sketch_rivers_flag = True
        sketch_basins_flag = True
        sketch_ridges_flag = True
        sketch_peaks_flag = True
    '''
    
    if sketch_rivers_flag == True:
        pixels_rivers = rivers.load() # create the pixel map
  
        for i in range(rivers.size[0]): # for every pixel:
            for j in range(rivers.size[1]):
                if pixels_rivers[i,j] == (255, 255, 255):
                    sketch[j,i] = (255, 0, 0)
    
    
    if sketch_ridges_flag == True:
        pixels_ridges = ridges.load() # create the pixel map
            
        for i in range(ridges.size[0]): # for every pixel:
            for j in range(ridges.size[1]):
                if pixels_ridges[i,j] == (255, 255, 255):
                    sketch[j,i] = (0, 0, 255) 
    
    
    if sketch_peaks_flag == True:
        pixels_peaks = peaks.load() # create the pixel map

        for i in range(peaks.size[0]): # for every pixel:
            for j in range(peaks.size[1]):
                if pixels_peaks[i,j] == (255, 255, 255):
                    sketch[j,i] = (0, 0, 0)   
            
    
    if sketch_basins_flag == True:
        pixels_basins = basins.load() # create the pixel map
            
        for i in range(basins.size[0]): # for every pixel:
            for j in range(basins.size[1]):
                if pixels_basins[i,j] == (255, 255, 255):
                    sketch[j,i] = (0, 255, 0)    
            
            
    #sketch_image = Image.fromarray(sketch)
    
    return sketch

In [5]:
files_len = len(os.listdir('heightmaps')) * 4
files_len_directory = len(os.listdir('heightmaps'))

idxs = [x for x in range(files_len)]
np.random.shuffle(idxs)

In [6]:
dim = (512, 512)

dataset_iterator = 0
iterator = 0

#f = h5py.File("dataset.hdf5", "w")
#f.create_dataset("sketches", (files_len, 512, 512, 3), dtype='float32')
#f.create_dataset("heightmaps", (files_len, 512, 512, 3), dtype='float32')
#f.create_dataset("textures", (files_len, 512, 512, 3), dtype='float32')

for iterator in range(files_len_directory):       
    print(str(dataset_iterator) + '/' + str(files_len))
    sketch = prepare_heightmap_sketch('heightmaps/topography_' + str(iterator+1) + '.tif')
    heightmap = cv2.imread('heightmaps/topography_' + str(iterator+1) + '.tif')
    #texture = imread('textures/word_' + str(iterator+1) + '.png')
    
    print('heightmaps/topography_' + str(iterator+1) + '.tif')
    #print('textures512/word_' + str(iterator+1) + '.png')
        
    #sketch_array = (sketch - 127.5) / 127.5
    #heightmap_array = (np.array(heightmap) - 127.5) / 127.5
    #texture_array = (np.array(texture) - 127.5) / 127.5
    
    sketch_array = np.array(sketch)
    heightmap_array = np.array(heightmap)
    
    # original rotation
    #f["sketches"][idxs[dataset_iterator]] = sketch_array
    #f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    #f["textures"][idxs[dataset_iterator]] = texture_array
    
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite("images/train/terrain_" + str(idxs[dataset_iterator]) + ".png",im_h)
    
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    #texture_array = np.rot90(texture_array)
        
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite("images/train/terrain_" + str(idxs[dataset_iterator]) + ".png",im_h)
        
    #f["sketches"][idxs[dataset_iterator]] = sketch_array
    #f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    #f["textures"][idxs[dataset_iterator]] = texture_array
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    #texture_array = np.rot90(texture_array)
        
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite("images/train/terrain_" + str(idxs[dataset_iterator]) + ".png",im_h)    
    
    #f["sketches"][idxs[dataset_iterator]] = sketch_array
    #f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    #f["textures"][idxs[dataset_iterator]] = texture_array
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    #texture_array = np.rot90(texture_array)
    
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite("images/train/terrain_" + str(idxs[dataset_iterator]) + ".png",im_h)
    
    #f["sketches"][idxs[dataset_iterator]] = sketch_array
    #f["heightmaps"][idxs[dataset_iterator]] = heightmap_array
    #f["textures"][idxs[dataset_iterator]] = texture_array
    dataset_iterator += 1

print(str(dataset_iterator) + '/' + str(files_len))
print("Done!")

0/22344
heightmaps/topography_1.tif
4/22344
heightmaps/topography_2.tif
8/22344
heightmaps/topography_3.tif
12/22344
heightmaps/topography_4.tif
16/22344
heightmaps/topography_5.tif
20/22344
heightmaps/topography_6.tif
24/22344
heightmaps/topography_7.tif
28/22344
heightmaps/topography_8.tif
32/22344
heightmaps/topography_9.tif
36/22344
heightmaps/topography_10.tif
40/22344
heightmaps/topography_11.tif
44/22344
heightmaps/topography_12.tif
48/22344
heightmaps/topography_13.tif
52/22344
heightmaps/topography_14.tif
56/22344
heightmaps/topography_15.tif
60/22344
heightmaps/topography_16.tif
64/22344
heightmaps/topography_17.tif
68/22344
heightmaps/topography_18.tif
72/22344
heightmaps/topography_19.tif
76/22344
heightmaps/topography_20.tif
80/22344
heightmaps/topography_21.tif
84/22344
heightmaps/topography_22.tif
88/22344
heightmaps/topography_23.tif
92/22344
heightmaps/topography_24.tif
96/22344
heightmaps/topography_25.tif
100/22344
heightmaps/topography_26.tif
104/22344
heightmaps/to

heightmaps/topography_209.tif
836/22344
heightmaps/topography_210.tif
840/22344
heightmaps/topography_211.tif
844/22344
heightmaps/topography_212.tif
848/22344
heightmaps/topography_213.tif
852/22344
heightmaps/topography_214.tif
856/22344
heightmaps/topography_215.tif
860/22344
heightmaps/topography_216.tif
864/22344
heightmaps/topography_217.tif
868/22344
heightmaps/topography_218.tif
872/22344
heightmaps/topography_219.tif
876/22344
heightmaps/topography_220.tif
880/22344
heightmaps/topography_221.tif
884/22344
heightmaps/topography_222.tif
888/22344
heightmaps/topography_223.tif
892/22344
heightmaps/topography_224.tif
896/22344
heightmaps/topography_225.tif
900/22344
heightmaps/topography_226.tif
904/22344
heightmaps/topography_227.tif
908/22344
heightmaps/topography_228.tif
912/22344
heightmaps/topography_229.tif
916/22344
heightmaps/topography_230.tif
920/22344
heightmaps/topography_231.tif
924/22344
heightmaps/topography_232.tif
928/22344
heightmaps/topography_233.tif
932/22344


heightmaps/topography_410.tif
1640/22344
heightmaps/topography_411.tif
1644/22344
heightmaps/topography_412.tif
1648/22344
heightmaps/topography_413.tif
1652/22344
heightmaps/topography_414.tif
1656/22344
heightmaps/topography_415.tif
1660/22344
heightmaps/topography_416.tif
1664/22344
heightmaps/topography_417.tif
1668/22344
heightmaps/topography_418.tif
1672/22344
heightmaps/topography_419.tif
1676/22344
heightmaps/topography_420.tif
1680/22344
heightmaps/topography_421.tif
1684/22344
heightmaps/topography_422.tif
1688/22344
heightmaps/topography_423.tif
1692/22344
heightmaps/topography_424.tif
1696/22344
heightmaps/topography_425.tif
1700/22344
heightmaps/topography_426.tif
1704/22344
heightmaps/topography_427.tif
1708/22344
heightmaps/topography_428.tif
1712/22344
heightmaps/topography_429.tif
1716/22344
heightmaps/topography_430.tif
1720/22344
heightmaps/topography_431.tif
1724/22344
heightmaps/topography_432.tif
1728/22344
heightmaps/topography_433.tif
1732/22344
heightmaps/topog

heightmaps/topography_610.tif
2440/22344
heightmaps/topography_611.tif
2444/22344
heightmaps/topography_612.tif
2448/22344
heightmaps/topography_613.tif
2452/22344
heightmaps/topography_614.tif
2456/22344
heightmaps/topography_615.tif
2460/22344
heightmaps/topography_616.tif
2464/22344
heightmaps/topography_617.tif
2468/22344
heightmaps/topography_618.tif
2472/22344
heightmaps/topography_619.tif
2476/22344
heightmaps/topography_620.tif
2480/22344
heightmaps/topography_621.tif
2484/22344
heightmaps/topography_622.tif
2488/22344
heightmaps/topography_623.tif
2492/22344
heightmaps/topography_624.tif
2496/22344
heightmaps/topography_625.tif
2500/22344
heightmaps/topography_626.tif
2504/22344
heightmaps/topography_627.tif
2508/22344
heightmaps/topography_628.tif
2512/22344
heightmaps/topography_629.tif
2516/22344
heightmaps/topography_630.tif
2520/22344
heightmaps/topography_631.tif
2524/22344
heightmaps/topography_632.tif
2528/22344
heightmaps/topography_633.tif
2532/22344
heightmaps/topog

heightmaps/topography_810.tif
3240/22344
heightmaps/topography_811.tif
3244/22344
heightmaps/topography_812.tif
3248/22344
heightmaps/topography_813.tif
3252/22344
heightmaps/topography_814.tif
3256/22344
heightmaps/topography_815.tif
3260/22344
heightmaps/topography_816.tif
3264/22344
heightmaps/topography_817.tif
3268/22344
heightmaps/topography_818.tif
3272/22344
heightmaps/topography_819.tif
3276/22344
heightmaps/topography_820.tif
3280/22344
heightmaps/topography_821.tif
3284/22344
heightmaps/topography_822.tif
3288/22344
heightmaps/topography_823.tif
3292/22344
heightmaps/topography_824.tif
3296/22344
heightmaps/topography_825.tif
3300/22344
heightmaps/topography_826.tif
3304/22344
heightmaps/topography_827.tif
3308/22344
heightmaps/topography_828.tif
3312/22344
heightmaps/topography_829.tif
3316/22344
heightmaps/topography_830.tif
3320/22344
heightmaps/topography_831.tif
3324/22344
heightmaps/topography_832.tif
3328/22344
heightmaps/topography_833.tif
3332/22344
heightmaps/topog

...

heightmaps/topography_4677.tif
18708/22344
heightmaps/topography_4678.tif
18712/22344
heightmaps/topography_4679.tif
18716/22344
heightmaps/topography_4680.tif
18720/22344
heightmaps/topography_4681.tif
18724/22344
heightmaps/topography_4682.tif
18728/22344
heightmaps/topography_4683.tif
18732/22344
heightmaps/topography_4684.tif
18736/22344
heightmaps/topography_4685.tif
18740/22344
heightmaps/topography_4686.tif
18744/22344
heightmaps/topography_4687.tif
18748/22344
heightmaps/topography_4688.tif
18752/22344
heightmaps/topography_4689.tif
18756/22344
heightmaps/topography_4690.tif
18760/22344
heightmaps/topography_4691.tif
18764/22344
heightmaps/topography_4692.tif
18768/22344
heightmaps/topography_4693.tif
18772/22344
heightmaps/topography_4694.tif
18776/22344
heightmaps/topography_4695.tif
18780/22344
heightmaps/topography_4696.tif
18784/22344
heightmaps/topography_4697.tif
18788/22344
heightmaps/topography_4698.tif
18792/22344
heightmaps/topography_4699.tif
18796/22344
heightmaps/

heightmaps/topography_4868.tif
19472/22344
heightmaps/topography_4869.tif
19476/22344
heightmaps/topography_4870.tif
19480/22344
heightmaps/topography_4871.tif
19484/22344
heightmaps/topography_4872.tif
19488/22344
heightmaps/topography_4873.tif
19492/22344
heightmaps/topography_4874.tif
19496/22344
heightmaps/topography_4875.tif
19500/22344
heightmaps/topography_4876.tif
19504/22344
heightmaps/topography_4877.tif
19508/22344
heightmaps/topography_4878.tif
19512/22344
heightmaps/topography_4879.tif
19516/22344
heightmaps/topography_4880.tif
19520/22344
heightmaps/topography_4881.tif
19524/22344
heightmaps/topography_4882.tif
19528/22344
heightmaps/topography_4883.tif
19532/22344
heightmaps/topography_4884.tif
19536/22344
heightmaps/topography_4885.tif
19540/22344
heightmaps/topography_4886.tif
19544/22344
heightmaps/topography_4887.tif
19548/22344
heightmaps/topography_4888.tif
19552/22344
heightmaps/topography_4889.tif
19556/22344
heightmaps/topography_4890.tif
19560/22344
heightmaps/

heightmaps/topography_5059.tif
20236/22344
heightmaps/topography_5060.tif
20240/22344
heightmaps/topography_5061.tif
20244/22344
heightmaps/topography_5062.tif
20248/22344
heightmaps/topography_5063.tif
20252/22344
heightmaps/topography_5064.tif
20256/22344
heightmaps/topography_5065.tif
20260/22344
heightmaps/topography_5066.tif
20264/22344
heightmaps/topography_5067.tif
20268/22344
heightmaps/topography_5068.tif
20272/22344
heightmaps/topography_5069.tif
20276/22344
heightmaps/topography_5070.tif
20280/22344
heightmaps/topography_5071.tif
20284/22344
heightmaps/topography_5072.tif
20288/22344
heightmaps/topography_5073.tif
20292/22344
heightmaps/topography_5074.tif
20296/22344
heightmaps/topography_5075.tif
20300/22344
heightmaps/topography_5076.tif
20304/22344
heightmaps/topography_5077.tif
20308/22344
heightmaps/topography_5078.tif
20312/22344
heightmaps/topography_5079.tif
20316/22344
heightmaps/topography_5080.tif
20320/22344
heightmaps/topography_5081.tif
20324/22344
heightmaps/

heightmaps/topography_5250.tif
21000/22344
heightmaps/topography_5251.tif
21004/22344
heightmaps/topography_5252.tif
21008/22344
heightmaps/topography_5253.tif
21012/22344
heightmaps/topography_5254.tif
21016/22344
heightmaps/topography_5255.tif
21020/22344
heightmaps/topography_5256.tif
21024/22344
heightmaps/topography_5257.tif
21028/22344
heightmaps/topography_5258.tif
21032/22344
heightmaps/topography_5259.tif
21036/22344
heightmaps/topography_5260.tif
21040/22344
heightmaps/topography_5261.tif
21044/22344
heightmaps/topography_5262.tif
21048/22344
heightmaps/topography_5263.tif
21052/22344
heightmaps/topography_5264.tif
21056/22344
heightmaps/topography_5265.tif
21060/22344
heightmaps/topography_5266.tif
21064/22344
heightmaps/topography_5267.tif
21068/22344
heightmaps/topography_5268.tif
21072/22344
heightmaps/topography_5269.tif
21076/22344
heightmaps/topography_5270.tif
21080/22344
heightmaps/topography_5271.tif
21084/22344
heightmaps/topography_5272.tif
21088/22344
heightmaps/

heightmaps/topography_5441.tif
21764/22344
heightmaps/topography_5442.tif
21768/22344
heightmaps/topography_5443.tif
21772/22344
heightmaps/topography_5444.tif
21776/22344
heightmaps/topography_5445.tif
21780/22344
heightmaps/topography_5446.tif
21784/22344
heightmaps/topography_5447.tif
21788/22344
heightmaps/topography_5448.tif
21792/22344
heightmaps/topography_5449.tif
21796/22344
heightmaps/topography_5450.tif
21800/22344
heightmaps/topography_5451.tif
21804/22344
heightmaps/topography_5452.tif
21808/22344
heightmaps/topography_5453.tif
21812/22344
heightmaps/topography_5454.tif
21816/22344
heightmaps/topography_5455.tif
21820/22344
heightmaps/topography_5456.tif
21824/22344
heightmaps/topography_5457.tif
21828/22344
heightmaps/topography_5458.tif
21832/22344
heightmaps/topography_5459.tif
21836/22344
heightmaps/topography_5460.tif
21840/22344
heightmaps/topography_5461.tif
21844/22344
heightmaps/topography_5462.tif
21848/22344
heightmaps/topography_5463.tif
21852/22344
heightmaps/

In [7]:
#f.close()